# Residual analysis

In [ ]:
using DatasetManager, LabDataSources, C3D, Biomechanics, DataFrames, Statistics, DSP, PlotlyJS, LinearAlgebra

In [ ]:
rootdir = # REDACTED
rawpath = joinpath(rootdir, "data", "parkinsons", "raw", "PD_ARMS_STAB")
genpath = joinpath(rootdir, "data", "parkinsons", "generated")

subsets = [
    DataSubset("c3d", Source{C3DFile}, rawpath, "Subject */_/*.c3d"),
];

In [ ]:
const markers = [
    "RUA3", "LTOE", "RASI", "XYPH", "RTH4", "RSK3", "LTH2", "LFAM", "RSHO", "C7", "T8", "LSHO",
    "LWRR", "RTH3", "LUA1", "LFAL", "LSK3", "RFAM", "L5MT", "RSK1", "RSK4", "LPSI", "RTH1",
    "RWRU", "LSK2", "LTH1", "RUA1", "RHEE", "RLHL", "LSK4", "RASI_2", "LSK1", "STRN", "LLHL",
    "LASI_2", "RUA2", "RWRR", "LWRU", "LTH3", "RTOE", "LASI", "R5MT", "LTH4", "LUA2", "RFAL",
    "LUA3", "RSK2", "RTH2", "LHEE", "RPSI"
];

## Characterize RMS noise of platform mounted markers

Three fixed markers are placed on the CAREN 6DOF platform. Since the markers are not labeled in these c3d files,
we use the heuristic that a motionless (i.e. platform mounted) marker exhibits a movement velocity less
than 0.1 mm/sec for at least 2.5 sec. We then calculate the RMS noise of the
matching markers. A limitation of this approach is that instability present in the platform will contribute
noise to the marker noise estimate, and the resulting RMS noise will be an overestimate of the RMS
noise of the motion capture markers. Therefore, we estimate the system noise from the static calibration trials when the participant is not moving.

In [ ]:
reftrials = let
    labels = Dict(:kind => [ r"[Ss]tatic[-_]?\d*"i, "relaxed"])
    conds = TrialConditions((:kind,), labels; subject_fmt=r"(?<=Subject )(?<subject>\d+\w?)")

    # Read all perturbations
    trials = findtrials(subsets, conds)
end
summarize(reftrials)

In [ ]:
refsrs = analyzedataset(reftrials, Source{C3DFile}) do trial
    seg = Segment(trial, "c3d")
    src = readsegment(seg; strip_prefixes=true)
    sr = SegmentResult(seg)
    res = results(sr)
    Fcs = collect(range(1.,30; length=250))
    
    foreach(filter(contains(r"^\*"), keys(src.point))) do mkr
        @views start = findfirst(!ismissing, src.point[mkr][:,1])
        @views finish = something(findnext(ismissing, src.point[mkr][:,1], start), lastindex(src.point[mkr], 1)+1)-1
        rg = start:finish
        
        if length(rg) > 250
            bs = fill(Inf, 3)
            as = zeros(3)
            mkrdata = convert(Matrix{Float32}, src.point[mkr][rg,:])
            for i in eachindex(bs)
                as[i], bs[i] = Biomechanics._linreg(totimes(rg, 100), mkrdata[:,i])
            end
            if norm(bs) < .1
                res[mkr] = Biomechanics.rmsd_simd(mkrdata, repeat(as', outer=length(rg)))
#                 res[mkr*"_slope"] = norm(bs)
#                 res[mkr*"_len"] = length(rg)
            end
        end
    end
    return sr
end

refdf = DatasetManager.stack(refsrs, [:kind])
describe(refdf)

In [ ]:
plot(histogram(;x=refdf[!,:value], histnorm="probability"),
    Layout(;width=800, height=400, xaxis_title="RMS Noise (mm)", yaxis_title="Probability",
        title="Histogram of RMS noise of stationary markers"))

Based on these results a baseline RMS noise of 0.5 is used for the succeeding residual analysis.

## Residual analysis of markers used in IK

In [ ]:
labels = Dict(:arms => [ ["NONE", "NA"] => "held", ["AS", "Norm", "NORM", "normal"] => "norm"],
                 :kind => [ ["(?<=NONE|NORM|held|norm)S", "BA", "single(?!task)"] => "singletask", ["(?<=NONE|NORM|norm|held)C", "CO", "CP", "dual(?!task)"] => "dualtask",
                           "PO" => "pert", ["PARK", "(?<=_)TR(?=_)"] => "park"],
                 :pert_side => [ ["[Rr](?=[ST]|(?i:slip|trip))", "RIGHT"] => "right", ["[Ll](?=[ST]|(?i:slip|trip))", "LEFT"] => "left"],
                 :pert_type => ["NP" => "steadystate", "(?i:(?<=[RL]|right|left))T" => "trip", "(?i:(?<=[RL]|right|left))S" => "slip"])
conds = TrialConditions((:arms,:kind,:pert_side,:pert_type), labels; required=(:arms,:kind,),
    subject_fmt=r"(?<=Subject |N|subject-)(?<subject>\d+\w?)",
    defaults=Dict(:pert_type => "steadystate", :pert_side => "NA"))

# Read all perturbations
trials = findtrials(subsets, conds;
    ignorefiles = [
        joinpath(rawpath, "Subject 08/_/held_single_ltrip.c3d"),
    ])
filter!(trials) do trial
    subject(trial) ∉ ("2", "06") &&
    conditions(trial)[:pert_type] == "steadystate" &&
    conditions(trial)[:arms] == "norm" &&
    conditions(trial)[:kind] ∈ ("singletask", "dualtask", "static", "digitize")
end
summarize(trials)

In [ ]:
srs = analyzedataset(trials, Source{C3DFile}; show_errors=false) do trial
    seg = Segment(trial, "c3d")
    src = readsegment(seg)
    sr = SegmentResult(seg)
    res = results(sr)
    Fcs = collect(range(1.,30; length=250))
    
    foreach(markers) do mkr
        fc, _ = optfc(convert(Matrix{Float32}, src.point[mkr][15*100:(end-10*100), :]), Fcs; fs=100, rmsnoise=0.5)
        if fc != 30
            res[mkr] = fc
        end
    end
    return sr
end

df = DatasetManager.stack(srs, conds)
show(describe(unstack(df), :mean, :min, :median, :max); allrows=true)